# Getting Started with Rocket Simulation Framework

Welcome! This notebook provides a 5-minute introduction to the rocket simulation framework.

## What You'll Learn

- How to load configurations
- Run your first rocket simulation
- Visualize flight trajectory
- Export results

## Prerequisites

Make sure you've installed all dependencies:

```bash
pip install -r requirements.txt
```

## Step 1: Imports

In [ ]:
import sys
from pathlib import Path

# Add src to path
sys.path.insert(0, str(Path.cwd().parent / 'src'))

from config_loader import ConfigLoader
from motor_builder import MotorBuilder
from environment_setup import EnvironmentBuilder
from rocket_builder import RocketBuilder
from flight_simulator import FlightSimulator
from visualizer import Visualizer

print("✓ Imports successful!")

## Step 2: Load Configuration

Configurations are stored in YAML files in the `configs/` directory.

In [ ]:
# Load configuration
config_path = Path('../configs/simple_rocket.yaml')

config_loader = ConfigLoader()
config_loader.load_from_yaml(str(config_path))

# Extract configurations
rocket_cfg = config_loader.get_rocket_config()
motor_cfg = config_loader.get_motor_config()
env_cfg = config_loader.get_environment_config()
sim_cfg = config_loader.get_simulation_config()

print(f"✓ Configuration loaded: {rocket_cfg.name}")
print(f"  Dry mass: {rocket_cfg.dry_mass_kg} kg")
print(f"  Fins: {rocket_cfg.fins.count} fins")
print(f"  Location: ({env_cfg.latitude_deg}°N, {env_cfg.longitude_deg}°E)")

## Step 3: Build Components

Use builder classes to create motor, environment, and rocket objects.

In [ ]:
# Build motor
motor_builder = MotorBuilder(motor_cfg)
motor = motor_builder.build()

print(f"✓ Motor built: {motor_cfg.thrust_source}")
print(motor_builder.get_summary())

In [ ]:
# Build environment
env_builder = EnvironmentBuilder(env_cfg)
environment = env_builder.build()

print("✓ Environment built")
print(env_builder.get_summary())

In [ ]:
# Build rocket
rocket_builder = RocketBuilder(rocket_cfg)
rocket = rocket_builder.build(motor)

print(f"✓ Rocket built: {rocket_cfg.name}")
print(rocket_builder.get_summary())

# Check stability
stability_info = rocket_builder.get_stability_info()
print(f"\n  Static margin: {stability_info['static_margin_calibers']:.2f} calibers")

## Step 4: Run Simulation

Execute the flight simulation!

In [ ]:
# Create flight simulator
simulator = FlightSimulator(
    rocket=rocket,
    motor=motor,
    environment=environment,
    rail_length_m=sim_cfg.rail_length_m,
    inclination_deg=sim_cfg.inclination_deg,
    heading_deg=sim_cfg.heading_deg
)

# Run simulation
print("Running simulation...")
flight = simulator.run()

print("✓ Simulation complete!")

## Step 5: View Results

In [ ]:
# Print flight summary
simulator.print_summary()

# Get metrics as dictionary
summary = simulator.get_summary()

print("\nKey Metrics:")
print(f"  Apogee: {summary['apogee_m']:.1f} m ({summary['apogee_m']/0.3048:.1f} ft)")
print(f"  Max velocity: {summary['max_velocity_ms']:.1f} m/s (Mach {summary['max_mach_number']:.2f})")
print(f"  Flight time: {summary['flight_time_s']:.1f} s")
print(f"  Landing distance: {summary['lateral_distance_m']:.1f} m")

## Step 6: Visualize Trajectory

In [ ]:
# Create visualizer
visualizer = Visualizer(flight)

# Create output directory
output_dir = Path('../outputs/getting_started')
output_dir.mkdir(parents=True, exist_ok=True)

# Generate plots
print("Generating plots...")

# 3D trajectory
visualizer.plot_trajectory_3d(
    output_path=str(output_dir / 'trajectory_3d.png')
)

# Altitude vs time
visualizer.plot_altitude_vs_time(
    output_path=str(output_dir / 'altitude_vs_time.png')
)

# Velocity vs time
visualizer.plot_velocity_vs_time(
    output_path=str(output_dir / 'velocity_vs_time.png')
)

print(f"✓ Plots saved to {output_dir}")

## Step 7: Export Data (Optional)

In [ ]:
from data_handler import DataHandler

# Create data handler
data_handler = DataHandler()

# Export trajectory to CSV
csv_path = output_dir / 'trajectory.csv'
data_handler.export_trajectory_csv(
    flight=flight,
    output_path=str(csv_path)
)

# Export summary to JSON
json_path = output_dir / 'summary.json'
data_handler.export_summary_json(
    summary=summary,
    output_path=str(json_path)
)

print(f"✓ Data exported:")
print(f"  CSV: {csv_path}")
print(f"  JSON: {json_path}")

## Success! 🚀

You've completed your first rocket simulation!

### What You Learned

1. ✓ Load configurations from YAML files
2. ✓ Build rocket components (motor, environment, rocket)
3. ✓ Run flight simulation
4. ✓ Analyze results
5. ✓ Generate visualizations
6. ✓ Export data

### Next Steps

Explore other notebooks:

- **01_single_flight_simulation.ipynb**: Detailed single flight workflow
- **02_monte_carlo_ensemble.ipynb**: Uncertainty quantification with Monte Carlo
- **03_sensitivity_analysis.ipynb**: Identify critical parameters
- **04_comparison_analysis.ipynb**: Compare multiple configurations

### Quick Tips

**Modify configurations**: Edit YAML files in `configs/` to test different rockets

**Try different motors**: Change `motor.thrust_source` in config file

**Vary environment**: Modify wind, location, or atmospheric model

**CLI scripts**: Use `scripts/run_single_simulation.py` for batch simulations

```bash
python scripts/run_single_simulation.py \
  --config configs/simple_rocket.yaml \
  --output-dir outputs/my_simulation
```

### Documentation

- Configuration schema: `configs/README_CONFIGS.md`
- Architecture: `docs/ARCHITECTURE.md`
- API reference: `docs/API_REFERENCE.md`

Happy simulating! 🎯